In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
#naming path for file retrieval
path = r'C:\Users\Admin\Documents\Data Analytics Course\Achievement 4\08-2020 Instacart Basket Analysis'

In [3]:
#import customers csv
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [4]:
customers.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [5]:
customers.shape

(206209, 10)

# Data Wrangling


In [6]:
#renaming columns for consistency
customers.rename(columns= {'user_id' : 'customer_id','First Name' : 'first_name', 'Surnam' : 'surname', 'Gender' :'gender', 'STATE' : 'state', 'Age' : 'age', 'date_joined' : 'date_joined', 'n_dependants' : 'no_of_dependants', 'fam_status' : 'marital_status'}, inplace = True)

In [7]:
customers

customer_id first_name   surname  gender           state  age  \
0             26711    Deborah  Esquivel  Female        Missouri   48   
1             33890   Patricia      Hart  Female      New Mexico   36   
2             65803    Kenneth    Farley    Male           Idaho   35   
3            125935   Michelle     Hicks  Female            Iowa   40   
4            130797        Ann   Gilmore  Female        Maryland   26   
...             ...        ...       ...     ...             ...  ...   
206204       168073       Lisa      Case  Female  North Carolina   44   
206205        49635     Jeremy   Robbins    Male          Hawaii   62   
206206       135902      Doris  Richmond  Female        Missouri   66   
206207        81095       Rose   Rollins  Female      California   27   
206208        80148    Cynthia     Noble  Female        New York   55   

       date_joined  no_of_dependants marital_status  income  
0         1/1/2017                 3        married  165665  
1         1/1/2017                 0         single   59285  
2         1/1/2017                 2        married   99568  
3         1/1/2017                 0         single   42049  
4         1/1/2017                 1        married   40374  
...            ...               ...            ...     ...  
206204    4/1/2020                 1        married  148828  
206205    4/1/2020                 3        married  168639  
206206    4/1/2020                 2        married   53374  
206207    4/1/2020                 1        married   99799  
206208    4/1/2020                 1        married   57095  

[206209 rows x 10 columns]

# Data Quality and Consistency Checks

In [8]:
#checking desciptive stats
customers.describe()

customer_id            age  no_of_dependants         income
count  206209.000000  206209.000000     206209.000000  206209.000000
mean   103105.000000      49.501646          1.499823   94632.852548
std     59527.555167      18.480962          1.118433   42473.786988
min         1.000000      18.000000          0.000000   25903.000000
25%     51553.000000      33.000000          0.000000   59874.000000
50%    103105.000000      49.000000          1.000000   93547.000000
75%    154657.000000      66.000000          3.000000  124244.000000
max    206209.000000      81.000000          3.000000  593901.000000

In [9]:
#checking data types
customers.dtypes

customer_id          int64
first_name          object
surname             object
gender              object
state               object
age                  int64
date_joined         object
no_of_dependants     int64
marital_status      object
income               int64
dtype: object

In [10]:
#changing customer_id data type to string
customers['customer_id'] = customers['customer_id'].astype('str')

In [11]:
#checking for mixed type data 
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)


first_name


The 'first_name' column has mixed type data.

In [12]:
#changing first_name column to string data type
customers['first_name'] = customers['first_name'].astype('str')

In [13]:
#checking for missing values 
customers.isnull().sum()

customer_id         0
first_name          0
surname             0
gender              0
state               0
age                 0
date_joined         0
no_of_dependants    0
marital_status      0
income              0
dtype: int64

No missing values

In [14]:
#check for duplicates
df_dups = customers[customers.duplicated()]

In [15]:
df_dups

Empty DataFrame
Columns: [customer_id, first_name, surname, gender, state, age, date_joined, no_of_dependants, marital_status, income]
Index: []

No duplicates

# Merging Customers df with other prepared data

In [16]:
#importing merged data frame
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_flags.pkl'))

In [17]:
ords_prods_merge.head()

order_id  customer_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329            1             1                   2                  8   
1   2398795            1             2                   3                  7   
2    473747            1             3                   3                 12   
3   2254736            1             4                   4                  7   
4    431534            1             5                   4                 15   

   days_since_prior_order  new_customer  product_id  add_to_cart_order  \
0                     NaN          True         196                  1   
1                    15.0         False         196                  1   
2                    21.0         False         196                  1   
3                    29.0         False         196                  1   
4                    28.0         False         196                  1   

   reordered  ...        price_label     busiest day    busiest_days  \
0          0  ...  Mid-range product  Regularly busy  Regularly busy   
1          1  ...  Mid-range product  Regularly busy    Slowest days   
2          1  ...  Mid-range product  Regularly busy    Slowest days   
3          1  ...  Mid-range product      Least busy    Slowest days   
4          1  ...  Mid-range product      Least busy    Slowest days   

   busiest_period_of_day  max_order  loyalty_flag mean_order spending_flag  \
0         Average orders         10  New customer   6.367797   Low Spender   
1         Average orders         10  New customer   6.367797   Low Spender   
2            Most orders         10  New customer   6.367797   Low Spender   
3         Average orders         10  New customer   6.367797   Low Spender   
4            Most orders         10  New customer   6.367797   Low Spender   

  median_since_prior_order         frequency_flag  
0                     20.5  Non-frequent customer  
1                     20.5  Non-frequent customer  
2                     20.5  Non-frequent customer  
3                     20.5  Non-frequent customer  
4                     20.5  Non-frequent customer  

[5 rows x 25 columns]

In [18]:
#checking ords_prods_merge data types
ords_prods_merge.dtypes

order_id                       int64
customer_id                    int64
order_number                   int64
orders_day_of_week             int64
order_hour_of_day              int64
days_since_prior_order       float64
new_customer                    bool
product_id                     int64
add_to_cart_order              int64
reordered                      int64
_merge                      category
product_name                  object
aisle_id                       int64
department_id                  int64
prices                       float64
price_label                   object
busiest day                   object
busiest_days                  object
busiest_period_of_day         object
max_order                      int64
loyalty_flag                  object
mean_order                   float64
spending_flag                 object
median_since_prior_order     float64
frequency_flag                object
dtype: object

In [19]:
#changing customers_id data type to string
ords_prods_merge['customer_id'] = ords_prods_merge['customer_id'].astype('str')

In [20]:
#merge ords_prods_merge with customers df
ords_prods_customers = ords_prods_merge.merge(customers, on = 'customer_id')

In [21]:
ords_prods_customers.head()

order_id customer_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329           1             1                   2                  8   
1   2398795           1             2                   3                  7   
2    473747           1             3                   3                 12   
3   2254736           1             4                   4                  7   
4    431534           1             5                   4                 15   

   days_since_prior_order  new_customer  product_id  add_to_cart_order  \
0                     NaN          True         196                  1   
1                    15.0         False         196                  1   
2                    21.0         False         196                  1   
3                    29.0         False         196                  1   
4                    28.0         False         196                  1   

   reordered  ...         frequency_flag first_name  surname  gender    state  \
0          0  ...  Non-frequent customer      Linda   Nguyen  Female  Alabama   
1          1  ...  Non-frequent customer      Linda   Nguyen  Female  Alabama   
2          1  ...  Non-frequent customer      Linda   Nguyen  Female  Alabama   
3          1  ...  Non-frequent customer      Linda   Nguyen  Female  Alabama   
4          1  ...  Non-frequent customer      Linda   Nguyen  Female  Alabama   

  age date_joined no_of_dependants marital_status  income  
0  31   2/17/2019                3        married   40423  
1  31   2/17/2019                3        married   40423  
2  31   2/17/2019                3        married   40423  
3  31   2/17/2019                3        married   40423  
4  31   2/17/2019                3        married   40423  

[5 rows x 34 columns]

In [22]:
#exporting ords_prods_customers 
ords_prods_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customers.pkl'))